In [ ]:
#
# This file contains the common functions and definitions used in the Egeria Hands on
# Lab Notebooks.
#

# These functions define the location and names of Coco Pharmaceutical's
# Open Metadata and Governance (OMAG) Server Platforms where the metadata servers
# and governance servers run.  The `os.environ.get` function tests for the presence
# of the environment variables that define the platform network addresses in the
# Docker and Kubernetes runtime environments.  If the environment variables are not
# present then the localhost defaults are used.

import os

corePlatformURL      = os.environ.get('corePlatformURL', 'http://localhost:8080')
corePlatformName     = "Core Platform"

dataLakePlatformURL  = os.environ.get('dataLakePlatformURL', 'http://localhost:8081')
dataLakePlatformName = "Data Lake Platform"

devPlatformURL       = os.environ.get('devPlatformURL', 'http://localhost:8082')
devPlatformName      = "Dev Platform"


# The OMAG Server Platforms host one to many OMAG Servers.  An OMAG Server could be
# a metadata server or a specialized governance server.  Its behavior is determined
# by a configuration document that defines which OMAG services are activated.
# All OMAG Server Platforms support the administration commands to define a server's
# configuration document.  It is also possible to create configuration documents
# through admin calls to one OMAG Server Platform and then deploy them to the
# OMAG Server Platform where they are to run.  In the Egeria hands on lab, the
# OMAG Server configuration is created on the dev platform and deployed to the
# production platforms as needed.

adminPlatformURL = devPlatformURL


# Gary Geeke is the IT Administration Lead at Coco Pharmaceuticals.
# He does all of the configuration for the OMAG Servers. Other users are introduced and make
# calls to the server as required

adminUserId   = "garygeeke"
petersUserId  = "peterprofile"
erinsUserId   = "erinoverview"
calliesUserId = 'calliequartile'

# These are the names of the metadata servers used by Coco Pharmaceuticals.  Each metadata
# server runs as an OMAG Server on one of the OMAG Server Platforms

cocoMDS1PlatformURL  = dataLakePlatformURL
cocoMDS1PlatformName = dataLakePlatformName
cocoMDS1Name         = "cocoMDS1"

cocoMDS2PlatformURL  = corePlatformURL
cocoMDS2PlatformName = corePlatformName
cocoMDS2Name         = "cocoMDS2"

cocoMDS3PlatformURL  = corePlatformURL
cocoMDS3PlatformName = corePlatformName
cocoMDS3Name         = "cocoMDS3"

cocoMDS4PlatformURL  = dataLakePlatformURL
cocoMDS4PlatformName = dataLakePlatformName
cocoMDS4Name         = "cocoMDS4"

cocoMDS5PlatformURL  = corePlatformURL
cocoMDS5PlatformName = corePlatformName
cocoMDS5Name         = "cocoMDS5"

cocoMDS6PlatformURL  = corePlatformURL
cocoMDS6PlatformName = corePlatformName
cocoMDS6Name         = "cocoMDS6"

cocoMDSxPlatformURL  = devPlatformURL
cocoMDSxPlatformName = devPlatformName
cocoMDSxName         = "cocoMDSx"


# These are the names of the governance servers used in Coco Pharmaceuticals' data lake.
# Each governance server runs as an OMAG Server on the Data Lake OMAG Server Platform.
# They also connect to a metadata server to retrieve their configuration and store their
# results.

findItDL01PlatformURL  = dataLakePlatformURL
findItDL01PlatformName = dataLakePlatformName
findItDL01Name         = "findItDL01"
findItDL01ServerType   = "Discovery Server"
findItDL01MDS          = "cocoMDS1"

fixItDL01PlatformURL   = dataLakePlatformURL
fixItDL01PlatformName  = dataLakePlatformName
fixItDL01Name          = "fixItDL01"
fixItDL01ServerType    = "Stewardship Server"
fixItDL01MDS           = "cocoMDS1"


#
# Common processing of REST API errors.
#

import requests
import pprint
import json

def printResponse(response):
    prettyResponse = json.dumps(response.json(), indent=4)
    print(" ")
    print("Response: ")
    print(prettyResponse)
    print(" ")
    
def printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response):
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            print("Unexpected response from server " + serverName)
            printResponse(response)
        else:
            exceptionErrorMessage = response.json().get('exceptionErrorMessage')
            exceptionSystemAction = response.json().get('exceptionSystemAction')
            exceptionUserAction   = response.json().get('exceptionUserAction')
            if exceptionErrorMessage != None:
                print(exceptionErrorMessage)
                print(" * " + exceptionSystemAction)
                print(" * " + exceptionUserAction)
            else:
                print("Unexpected response from server " + serverName)
                printResponse(response)
    else:
        print("Unexpected response from server platform " + serverPlatformName + " at " + serverPlatformURL)
        printResponse(response)


#
# Metadata server configuration functions.  These functions add definitions
# to an OMAG server's configuration document
#

def postAndPrintResult(url, json=None, headers=None):
    print("   ...... (POST", url, ")")
    response = requests.post(url, json=json, headers=headers)
    print("   ...... Response: ", response.json())

def configurePlatformURL(adminPlatformURL, adminUserId, serverName, serverPlatform):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print("   ... configuring the platform the server will run on...")
    url = adminCommandURLRoot + serverName + '/server-url-root?url=' + serverPlatform
    postAndPrintResult(url)

def configureMaxPageSize(adminPlatformURL, adminUserId, serverName, maxPageSize):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the maximum page size...")
    url = adminCommandURLRoot + serverName + '/max-page-size?limit=' + maxPageSize
    postAndPrintResult(url)

def configureServerType(adminPlatformURL, adminUserId, serverName, serverType):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName=' + serverType
    postAndPrintResult(url)
    
def clearServerType(adminPlatformURL, adminUserId, serverName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... clearing the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName='
    postAndPrintResult(url)

def configureOwningOrganization(adminPlatformURL, adminUserId, serverName, organizationName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's owning organization...")
    url = adminCommandURLRoot + serverName + '/organization-name?name=' + organizationName
    postAndPrintResult(url)

def configureUserId(adminPlatformURL, adminUserId, serverName, userId):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's userId...")
    url = adminCommandURLRoot + serverName + '/server-user-id?id=' + userId
    postAndPrintResult(url)

def configurePassword(adminPlatformURL, adminUserId, serverName, password):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's password (optional)...")
    url = adminCommandURLRoot + serverName + '/server-user-password?password=' + password
    postAndPrintResult(url)

def configureSecurityConnection(adminPlatformURL, adminUserId, serverName, securityBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's security connection...")
    url = adminCommandURLRoot + serverName + '/security/connection'
    jsonContentHeader = {'content-type':'application/json'}
    postAndPrintResult(url, json=securityBody, headers=jsonContentHeader)

def configureDefaultAuditLog(adminPlatformURL, adminUserId, serverName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the default audit log...")
    url = adminCommandURLRoot + serverName + '/audit-log-destinations/default'
    postAndPrintResult(url)
    
def configureMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryType):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the metadata repository...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/' + repositoryType
    postAndPrintResult(url)
    
def configureRepositoryProxyDetails(adminPlatformURL, adminUserId, serverName, connectorProvider):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the repository proxy...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/repository-proxy/details?connectorProvider=' + connectorProvider
    postAndPrintResult(url)

def configureDescriptiveName(adminPlatformURL, adminUserId, serverName, collectionName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the short descriptive name of the metadata stored in this server...")
    url = adminCommandURLRoot + serverName + '/local-repository/metadata-collection-name/' + collectionName
    postAndPrintResult(url)

def configureEventBus(adminPlatformURL, adminUserId, serverName, busBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the event bus for this server...")
    url = adminCommandURLRoot + serverName + '/event-bus'
    postAndPrintResult(url, json=busBody, headers=jsonContentHeader)

def configureCohortMembership(adminPlatformURL, adminUserId, serverName, cohortName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the membership of the cohort...")
    url = adminCommandURLRoot + serverName + '/cohorts/' + cohortName
    postAndPrintResult(url)

def configureAccessService(adminPlatformURL, adminUserId, serverName, accessService, accessServiceOptions):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + accessService + " access service for this server...")
    url = adminCommandURLRoot + serverName + '/access-services/' + accessService
    jsonContentHeader = {'content-type':'application/json'}
    postAndPrintResult(url, json=accessServiceOptions, headers=jsonContentHeader)

def configureDiscoveryEngineServices(adminPlatformURL, adminUserId, discoServerName, mdrServerName, mdrServerPlatform, discoveryEngines):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + discoServerName + " discovery engine services...")
    jsonContentHeader = {'content-type':'application/json'}
    accessServiceClient = {
        "class": "OMAGServerClientConfig",
        "omagserverPlatformRootURL": mdrServerPlatform,
        "omagserverName" : mdrServerName
    }
    print(accessServiceClient)
    discoveryEngineServicesURLRoot = adminCommandURLRoot + discoServerName + '/discovery-engine-services'
    url = discoveryEngineServicesURLRoot + '/client-config'
    postAndPrintResult(url, json=accessServiceClient, headers=jsonContentHeader)
    url = discoveryEngineServicesURLRoot + '/discovery-engines'
    postAndPrintResult(url, json=discoveryEngines, headers=jsonContentHeader)
 

In [ ]:
# Server configuration can be deployed from one OMAG Server Platform to another.
# This enables a server configuration to be tested and then deployed in production.

def deployServerToPlatform(adminPlatformURL, adminUserId, serverName, platformURL):
    print("   ... deploying", serverName, "to the", platformURL, "platform...")
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    url = adminCommandURLRoot + serverName + '/configuration/deploy'
    platformTarget = {
        "class": "URLRequestBody",
        "urlRoot": platformURL
    }
    jsonContentHeader = {'content-type':'application/json'}
    postAndPrintResult(url, json=platformTarget, headers=jsonContentHeader)


In [ ]:
# The OMAG Server Platform is a single executable (application) that can be started
# from the command line or a script or as part of a pre-built container environment
# such as docker-compose or kubernetes.  The function below checks that a specific
# server platform is running.

def checkServerPlatform(serverPlatformName, serverPlatformURL):
    try:
        isPlatformActiveURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/origin"
        response = requests.get(isPlatformActiveURL)
        if response.status_code == 200:
            print("   ", serverPlatformName, "is active")
            return True
        else:
            print("   ", serverPlatformName, "is not an OMAG Platform")
            return False
    except Exception as error:
        print("Exception: %s" % error)
        print("   ", serverPlatformName, "is down - start it before proceeding")
        return False


# The OMAG Server Platform has the implementation of the open metadata and governance (OMAG) services.  These are
# divided into three groups: Common Services, Access Services and Governance Services.

def printServiceDescriptions(serverPlatformName, serviceGroupName, services):
    print(serviceGroupName, "for", serverPlatformName)
    for x in range(len(services)):
        serviceName = services[x].get('serviceName')
        serviceDescription = services[x].get('serviceDescription')
        serviceWiki = services[x].get('serviceWiki')    
        print (" * ", serviceName)
        print ("     ", serviceDescription)
        print ("     ", serviceWiki)
    print (" ")
    
def getServerPlatformServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = requests.get(getOMAGServicesURL + "/common-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Common services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/access-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Access services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/governance-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Governance services", response.json().get('services'))

In [ ]:

# Each server is configured to define which services should be actived.  This configuration results in
# the creation of a configuration document.  This document is read when the server is started and
# drives the initialization of the services.

def checkServerConfigured(serverName, serverPlatformName, serverPlatformURL):
    isServerKnownOnPlatform = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + "/servers/" + serverName + "/configuration"
    response = requests.get(isServerKnownOnPlatform)
    if response.status_code == 200:
        serverConfig=response.json().get('omagserverConfig')
        auditTrail = serverConfig.get('auditTrail')
        if auditTrail is not None:
            print("           ...", serverName, "is configured")
            return True
        else:
            print("           ...", serverName, "needs configuring")
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False


# The OMAG Server Platform supports a call to return if a server is active.

def checkServerActive(serverName, serverPlatformName, serverPlatformURL):
    isServerActiveOnPlatform = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/status"
    response = requests.get(isServerActiveOnPlatform)
    if response.status_code == 200:
        serverStatus = response.json().get('active')
        if serverStatus == True:
            print("           ...", serverName, "is active - ready to begin")
        else:
            print("           ...", serverName, "is down - needs to be started")
        return serverStatus
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False

    
# This is the call to start a server on a specific platform.  Once the server is running it is possible to
# make use of the open metadata and governance services that have been activated in the server.

def activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL):
    print ("                Starting server " + serverName + " ...")
    activateServerURL = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + '/servers/' + serverName + "/instance"
    response = requests.post(activateServerURL)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            return True
        else:
            errorMessage = response.json().get('exceptionErrorMessage')
            print ("                   ..." + serverName + " failed to start")
            print ("                       error message is: " + errorMessage)
            return False
                
    else:
        print ("                   ..." + serverName + " failed to start")
        return False


# Once a server is active, it is possible to query the services that are active.

def getServerServices(serverName, serverPlatformName, serverPlatformURL):
    getServerActiveServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/services"
    print ("Services for server:", serverName)
    response = requests.get(getServerActiveServicesURL)
    if response.status_code == 200:
        serviceList = response.json().get('serverServicesList')
        for x in range(len(serviceList)):
            print (" * ", serviceList[x])
    else:
        print (response)

# This function checks whether a server is active and starts it if it is down.

def activateServerIfDown(serverName, serverPlatformName, serverPlatformURL):
    print("        Checking OMAG Server " + serverName)
    configured = checkServerConfigured(serverName, serverPlatformName, serverPlatformURL)
    if configured == True:
        active = checkServerActive(serverName, serverPlatformName, serverPlatformURL)
        if active == False:
            activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL)

# This function checks the platform is running and ensures the servers are started on it.
# It requests user action if either the platform is not running or the servers are not configured.
# Otherwise it should return with all of the servers running.

def activatePlatform(serverPlatformName, serverPlatformURL, hostedServerNames):
    available = checkServerPlatform(serverPlatformName, serverPlatformURL)
    if available == True:
        for x in range(len(hostedServerNames)):
            activateServerIfDown(hostedServerNames[x], serverPlatformName, serverPlatformURL)
   

In [ ]:

# The open metadata servers are linked together through the following open metadata cohorts.
# The servers linked via a cohort can exchange open metadata either through federated
# queries or metadata replication.

cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"
ctsCohort  = "ctsCohort"

def queryServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    cohortNames = []
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts != None:
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        cohortNames.append(cohortName)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    return cohortNames


def printServerCohortsStatus(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts == None:
                    print("Server " + serverName + " is not connected to any cohorts")
                else:
                    print("Server " + serverName + " is connected to the following cohorts:")
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        connectionStatus = cohorts[x].get('connectionStatus')
                        print (" * " + cohortName + " [" + connectionStatus + "]")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printCohortMember(cohortMember, localRegistration):
    serverName = cohortMember.get('serverName')
    serverType = cohortMember.get('serverType')
    metadataCollectionId = cohortMember.get('metadataCollectionId')
    metadataCollectionName = cohortMember.get('metadataCollectionName')
    registrationTime = cohortMember.get('registrationTime')
    if localRegistration == True:
        print("Registration details for local " + serverType + " " + serverName)
    else:    
        print("Registration details for remote " + serverType + " " + serverName)
    if (metadataCollectionId != None):
        print(" * Metadata collection id:   " + metadataCollectionId)
    if (metadataCollectionName != None):
        print(" * Metadata collection name: " + metadataCollectionName)
    if (registrationTime != None):
        print(" * Registration time:        " + registrationTime)
    repositoryConnection = cohortMember.get('repositoryConnection')
    if repositoryConnection != None:
        endpoint = repositoryConnection.get('endpoint')
        if endpoint != None:
            address = endpoint.get('address')
            if address != None:
                print(" * URL for metadata queries: " + address)
            else:
                print(" * URL for metadata queries: null")
        else:
            print(" * URL for metadata queries: no endpoint")
    else:
        print(" * URL for metadata queries: not supported")


def printLocalRegistration(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/local-registration'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printLocalRegistrationForCohort(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/local-registration'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printRemoteRegistrations(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore = '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/remote-members'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMembers = response.json().get('cohortMembers')
                if cohortMembers != None:
                    for x in range(len(cohortMembers)):
                        printCohortMember(cohortMembers[x], False)
                else:
                    print("No remote members")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    

def printServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    print("Reviewing the cohort registry for server: " + serverName)
    print(" ")
    printLocalRegistration(serverName, serverPlatformName, serverPlatformURL)
    print(" ")
    cohorts = queryServerCohorts(serverName, serverPlatformName, serverPlatformURL)
    for x in range(len(cohorts)):
        print("Cohort " + cohorts[x] + " member details")
        printLocalRegistrationForCohort(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        printRemoteRegistrations(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        print(" ")
        

In [ ]:
#
# Working with assets - this first set of functions displays assets returned from the open metadata repositories.
# 

def printAssetSummary(asset):
    elementType = asset.get('type')
    assetType = elementType.get('elementTypeName')
    assetGUID = asset.get('guid')
    assetQualifiedName = asset.get('qualifiedName')
    print(assetType + " \t| " + assetGUID + " | " + assetQualifiedName)
    
def printAssetDetail(asset):
    elementType = asset.get('type')
    assetType = elementType.get('elementTypeName')
    assetSuperTypes = elementType.get('elementSuperTypeNames')
    assetGUID = asset.get('guid')
    assetQualifiedName = asset.get('qualifiedName')
    assetDisplayName = asset.get('displayName')
    assetDescription = asset.get('description')
    assetOwner = asset.get('owner')
    assetOwnerType = asset.get('ownerType')
    assetZones = asset.get('zoneMembership')
    assetLatestChange = asset.get('latestChange')
    print(assetDisplayName + " [" + assetGUID + "]")
    print("  qualifiedName: " + assetQualifiedName)
    if assetDescription:
        print("  description:   " + assetDescription)
    print("  type:          " + assetType + ' [%s]' % ', '.join(map(str, assetSuperTypes)))
    if assetOwner:
        print("  owner:         " + assetOwner + " [" + assetOwnerType + "]")
    if assetZones:
        print("  zones:         " + '%s' % ', '.join(map(str, assetZones)))
    if assetLatestChange:
        print("  latest change: " + assetLatestChange)


def printAssetListSummary(assets):
    print(" ")
    for x in range(len(assets)):
        printAssetSummary(assets[x])

def printAssetListDetail(assets):
    print("\n--------------------------------------\n")
    for x in range(len(assets)):
        printAssetDetail(assets[x])
        print("\n--------------------------------------\n")


def printGUIDList(guids):
    if guids == None:
        print("No assets created")
    else:
        prettyGUIDs = json.dumps(guids, indent=4)
        print(prettyGUIDs)

def getLastGUID(guids):
    if guids == None:
        return "<unknown>"
    else:
        for guid in guids:
            returnGUID = guid
        return returnGUID


#
# Using the Asset Owner OMAS interface to create and query assets.  Notice that the interface returns all of the asset contents.
#

def assetOwnerCreateAssetWithDebug(assetTypeURL, serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    createAssetURL = assetOwnerURL + assetTypeURL
    jsonHeader = {'content-type':'application/json'}
    createAssetBody = {
        "class" : "NewCSVFileAssetRequestBody",
        "displayName" : displayName,
        "description" : description,
        "fullPath" : fullPath
    }
    print (" ")
    print ("POST " + createAssetURL)
    prettyBody = json.dumps(createAssetBody, indent=4)
    print (prettyBody)
    print (" ")
    response=requests.post(createAssetURL, json=createAssetBody, headers=jsonHeader)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")

def assetOwnerCreateCSVAssetWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerCreateAssetWithDebug('/assets/data-files/csv', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)

def assetOwnerCreateAvroAssetWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerCreateAssetWithDebug('/assets/data-files/avro', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)

    
def assetOwnerCreateAsset(assetTypeURL, serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    createAssetURL = assetOwnerURL + assetTypeURL
    jsonHeader = {'content-type':'application/json'}
    createAssetBody = {
        "class" : "NewCSVFileAssetRequestBody",
        "displayName" : displayName,
        "description" : description,
        "fullPath" : fullPath
    }
    response=requests.post(createAssetURL, json=createAssetBody, headers=jsonHeader)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets created")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        return []

def assetOwnerCreateCSVAsset(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    return assetOwnerCreateAsset('/assets/data-files/csv', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)

def assetOwnerCreateAvroAsset(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    return assetOwnerCreateAsset('/assets/data-files/avro', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)
    

def assetOwnerSearchForAssetsWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    getAssetsURL = assetOwnerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    print (" ")
    print ("POST " + getAssetsURL)
    print ("{ " + searchString + " }")
    print (" ")
    response=requests.post(getAssetsURL, data=searchString)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")
    if response.json().get('assets'):
        if len(response.json().get('assets')) == 1:
            print ("1 asset found")
        else:
            print (str(len(response.json().get('assets'))) + " assets found")
    else:
        print ("No assets found")



def assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    getAssetsURL = assetOwnerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    response = requests.post(getAssetsURL, data=searchString)
    assets = response.json().get('assets')
    if assets:
        return assets
    else:
        print ("No assets found")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def assetOwnerPrintAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets:
        if len(assets) == 1:
            print ("1 asset found:")
        else:
            print (str(len(assets)) + " assets found:")
        printAssetListSummary(assets)
        printAssetListDetail(assets)
    

def addOwner(serverName, serverPlatformName, serverPlatformURL, userId, assetName, assetGUID, assetOwner, ownerType):
    governanceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId 
    print ("Setting owner on " + assetName + " to " + assetOwner + " ...")
    body = {
        "class" : "OwnerRequestBody",
        "ownerType" : ownerType,
        "ownerId" : assetOwner
    }
    jsonHeader = {'content-type':'application/json'}
    addOwnerURL = governanceURL + "/assets/" + assetGUID + "/owner"
    response=requests.post(addOwnerURL, json=body, headers=jsonHeader)
    if response.status_code != 200:
        prettyResponse = json.dumps(response.json(), indent=4)
        print ("Response: ")
        print (prettyResponse)
        print (" ")

        
def addZones(serverName, serverPlatformName, serverPlatformURL, userId, assetName, assetGUID, zones):
    print ("Setting governance zones on " + assetName + " ...")
    governanceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId 
    jsonHeader = {'content-type':'application/json'}
    addZonesURL = governanceURL + "/assets/" + assetGUID + "/governance-zones"
    response=requests.post(addZonesURL, json=zones, headers=jsonHeader)
    if response.status_code != 200:
        prettyResponse = json.dumps(response.json(), indent=4)
        print ("Response: ")
        print (prettyResponse)
        print (" ")

#
# Working with the Asset Consumer OMAS.  This API returns lists of GUIDs or individual guids for gets and finds.  Then
# There are API operations to retrieve the corresponding Assets at different levels of detail.
#
def assetConsumerSearchForAssetsWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    getAssetsURL = assetConsumerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    print (" ")
    print ("GET " + getAssetsURL)
    print ("{ " + searchString + " }")
    print (" ")
    response=requests.post(getAssetsURL, data=searchString)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")
    if response.json().get('guids'):
        if len(response.json().get('guids')) == 1:
            print ("1 asset found")
        else:
            print (str(len(response.json().get('guids'))) + " GUIDs for matching assets found")
    else:
        print ("No assets found")

def assetConsumerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    getAssetsURL = assetConsumerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    response=requests.post(getAssetsURL, data=searchString)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets found")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def assetConsumerPrintAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    getAssetsURL = assetConsumerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    response=requests.post(getAssetsURL, data=searchString)
    guids = assetConsumerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if guids:
        printGUIDList(guids)


def addCommentToAssetWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    addCommentURL = assetConsumerURL + '/assets/' + assetGUID + '/comments'
    print("")
    print ("POST " + addCommentURL)
    jsonHeader = {'content-type':'application/json'}
    commentBody={
        "class" : "CommentRequestBody",
        "isPublic" : isPublic,
        "commentType" : commentType,
        "commentText" : commentText
    }
    prettyBody = json.dumps(commentBody, indent=4)
    print(prettyBody)
    response = requests.post(addCommentURL, json=commentBody, headers=jsonHeader)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")


def addCommentToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    addCommentURL = assetConsumerURL + '/assets/' + assetGUID + '/comments'
    jsonHeader = {'content-type':'application/json'}
    commentBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    response = requests.post(addCommentURL, json=commentBody, headers=jsonHeader)
    commentGUID = response.json().get('guid')
    if commentGUID:
        return commentGUID
    else:
        print ("No comment added")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def printAssetCommentsWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    commentQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments?elementStart=0&maxElements=50'
    print (" ")
    print ("GET " + commentQuery)
    response = requests.get(commentQuery)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")


def printComment(commentObject):
    if commentObject:
        comment   = commentObject.get('comment')
        relyCount = commentObject.get('replyCount')
        if comment:
            commentType = comment.get('commentType')
            commentText = comment.get('commentText') 
            commentUser = comment.get('user') 
            isPublic    = comment.get('isPublic') 
            if commentType:
                print("  comment type: " + commentType)
            if commentText:
                print("  comment text: " + commentText)
            if commentUser:
                print("  created by:   " + commentUser)
            if isPublic:
                print("  public:       " + isPublic)
    

def printCommentList(responseObjects):
    print(" ")
    if len(responseObjects) == 0:
        print ("No comments found")
    else:
        if len(responseObjects) == 1:
            print ("1 comment found:")
        else:
            print (str(len(responseObjects)) + " comments found:")
        for x in range(len(responseObjects)):
            printComment(responseObjects[x]) 


def printAssetComments(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    commentQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments?elementStart=0&maxElements=50'
    response = requests.get(commentQuery)
    responseObjects = response.json().get('list')
    if (responseObjects):
        printCommentList(responseObjects)
    else:
        print("No comments returned")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def addReplyToAssetCommentWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    commentReplyURL = assetConsumerURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies'
    print (" ")
    print ("POST " + commentReplyURL)
    commentReplyBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    jsonHeader = {'content-type':'application/json'}
    response = requests.post(commentReplyURL, json=commentReplyBody, headers=jsonHeader)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")
    

def addReplyToAssetComment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    commentReplyURL = assetConsumerURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies'
    print (" ")
    print ("POST " + commentReplyURL)
    commentReplyBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    jsonHeader = {'content-type':'application/json'}
    response = requests.post(commentReplyURL, json=commentReplyBody, headers=jsonHeader)
    commentGUID = response.json().get('guid')
    if commentGUID:
        return commentGUID
    else:
        print ("No comment added")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def printAssetCommentRepliesWithDebug(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    commentReplyQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies?elementStart=0&maxElements=50'
    print (" ")
    print ("GET " + commentReplyQuery)
    response = requests.get(commentReplyQuery)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")
    
def printAssetCommentReplies(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    commentReplyQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies?elementStart=0&maxElements=50'
    response = requests.get(commentReplyQuery)
    responseObjects = response.json().get('list')
    if (responseObjects):
        printCommentList(responseObjects)
    else:
        print("No comments returned")
        if response.status_code != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def printAssetUniverse(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    getAsset = connectedAssetURL + '/assets/' + assetGUID
    print (" ")
    print ("GET " + getAsset)
    response = requests.get(getAsset)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")
    
def printRelatedAssets(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + userId 
    getRelatedAsset = connectedAssetURL + '/assets/' + assetGUID + '/related-assets?elementStart=0&maxElements=50'
    print (" ")
    print ("GET " + getRelatedAsset)
    response = requests.get(getRelatedAsset)
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")


In [ ]:
# Open Discovery
# Working with the Discovery server and the Discovery Engine OMAS of the metadata server.


# The configuration of discovery engines and discovery services is managed in an open metadata server
# through the Discovery Engine OMAS.  The functions below support the definition of these capabilities.


def getDiscoveryEngineProperties(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/discovery-engine/users/" + userId
    getDiscoveryEngineURL = configCommandURLRoot + '/discovery-engines/by-name/' + qualifiedName
    discoveryEngineProperties = None
    response = requests.get(getDiscoveryEngineURL)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            discoveryEngineProperties = response.json().get('discoveryEngineProperties')
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return discoveryEngineProperties


def createDiscoveryEngine(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName, displayName, description):
    discoveryEngineProperties = getDiscoveryEngineProperties(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName)
    if (discoveryEngineProperties):
        print("Discovery Engine " + qualifiedName + " is already defined with the following properties:")
        displayName = discoveryEngineProperties.get('displayName')
        if (displayName):
            print("   displayName: " + displayName)
        description = discoveryEngineProperties.get('description')
        if (description):
            print("   description: " + description)
        return discoveryEngineProperties.get('guid')
    else:
        configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/discovery-engine/users/" + userId
        createDiscoveryEngineURL = configCommandURLRoot + '/discovery-engines'
        discoveryEngineGUID = None
        try:
            jsonHeader = {'content-type':'application/json'}
            body = {
                "class" : "NewDiscoveryEngineRequestBody",
                "qualifiedName" : qualifiedName,
                "displayName" : displayName,
                "description" : description
            }
            response=requests.post(createDiscoveryEngineURL, json=body, headers=jsonHeader)
            if response.status_code == 200:
                relatedHTTPCode = response.json().get('relatedHTTPCode')
                if relatedHTTPCode == 200:
                    discoveryEngineGUID = response.json().get('guid')
                else:
                    printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        except Exception as error:
            print("Exception: %s" % error)
            print("Platform " + serverPlatformURL + " is returning an error")
        return discoveryEngineGUID


def createDiscoveryService(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName, displayName, description, providerClassName):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/discovery-engine/users/" + userId        
    createDiscoveryServiceURL = configCommandURLRoot + '/discovery-services'
    discoveryServiceGUID = None
    try:
        jsonHeader = {'content-type':'application/json'}
        body = {
            "class" : "NewDiscoveryServiceRequestBody",
            "qualifiedName" : qualifiedName,
            "displayName" : displayName,
            "description" : description,
            "connection" : {
                "class": "Connection",
                "type": {
                    "class": "ElementType",
                    "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
                    "elementTypeName": "Connection",
                    "elementTypeVersion": 1,
                    "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
                    "elementOrigin": "CONFIGURATION"
                },
                "qualifiedName": qualifiedName + "-implementation",
                "displayName": displayName + " Implementation Connection",
                "description": "Connection for discovery service " + qualifiedName,
                "connectorType": {
                    "class": "ConnectorType",
                    "type": {
                        "class": "ElementType",
                        "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
                        "elementTypeName": "ConnectorType",
                        "elementTypeVersion": 1,
                        "elementTypeDescription": "A set of properties describing a type of connector.",
                        "elementOrigin": "LOCAL_COHORT"
                    },
                    "guid": "1111f73d-e343-abcd-82cb-3918fed81da6",
                    "qualifiedName": qualifiedName + "-DiscoveryServiceProvider",
                    "displayName": displayName + " Discovery Service Provider Implementation",
                    "description": description,
                    "connectorProviderClassName": providerClassName
                }
            }
        }
        response=requests.post(createDiscoveryServiceURL, json=body, headers=jsonHeader)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                discoveryServiceGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
    return discoveryServiceGUID


def registerDiscoveryServiceWithEngine(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineGUID, discoveryServiceGUID, discoveryRequestTypes):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/discovery-engine/users/" + userId        
    registerDiscoveryServiceURL = configCommandURLRoot + '/discovery-engines/' + discoveryEngineGUID + '/discovery-services'
    discoveryServerGUID = None
    try:
        jsonHeader = {'content-type':'application/json'}
        body = {
            "class" : "DiscoveryServiceRegistrationRequestBody",
            "discoveryServiceGUID" : discoveryServiceGUID,
            "discoveryRequestTypes" : discoveryRequestTypes
        }
        response=requests.post(registerDiscoveryServiceURL, json=body, headers=jsonHeader)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                discoveryServiceGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")


# The Discover Server periodically refreshes its configuration.  This call causes it to request fresh
# configuration from the Metadata Server immediately.  This is useful in demos or labs so that there is
# no need to wait for the asynchronous update if demonstrating discovery server configuration and you want
# any configuration changes to take effect immediately.

def refreshDiscoveryEngineConfig(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName):
    discoveryServerCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata//discovery-server/users/" + userId
    refreshConfigURL = discoveryServerCommandURLRoot + '/discovery-engines/' + qualifiedName + '/refresh-config'
    response = requests.get(refreshConfigURL)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode != 200:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

# The discovery server can return the current status of the discovery engines that it is hosting.
# This is useful to show how the discovery server is configured and the differnet types of
# discovery requests that are supported.

def printDiscoveryEngineStatus(discoveryEngineSummary):
    discoveryEngineName = discoveryEngineSummary.get('discoveryEngineName')
    if discoveryEngineName:
        print("  Discovery Engine: " + discoveryEngineName)
        discoveryEngineStatus = discoveryEngineSummary.get('discoveryEngineStatus')
        if discoveryEngineStatus:
            print("     status:       " + discoveryEngineStatus)
        discoveryEngineGUID = discoveryEngineSummary.get('discoveryEngineGUID')
        if discoveryEngineGUID:
            print("     guid:         " + discoveryEngineGUID)
        discoveryEngineDescription = discoveryEngineSummary.get('discoveryEngineDescription')
        if discoveryEngineDescription:
            print("     description:  " + discoveryEngineDescription)
        discoveryRequestTypes = discoveryEngineSummary.get('discoveryRequestTypes')
        if discoveryRequestTypes:
            print("     requestTypes: ")
            for x in range(len(discoveryRequestTypes)):
                print("       - " + discoveryRequestTypes[x])

def getDiscoveryEngineStatuses(serverName, serverPlatformName, serverPlatformURL, userId):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/discovery-server/users/' + userId
        getStatusURL = discoveryServerRootURL + '/discovery-engines/status'
        response = requests.get(getStatusURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                discoveryEngineSummaries = response.json().get('discoveryEngineSummaries')
                if discoveryEngineSummaries:
                    if len(discoveryEngineSummaries) == 1:
                        print("One discovery engine defined for discovery server " + serverName)
                    else:
                        print(str(len(discoveryEngineSummaries)) + " discovery engines defined for discovery server " + serverName)
                    print(' ')
                    for x in range(len(discoveryEngineSummaries)):
                        printDiscoveryEngineStatus(discoveryEngineSummaries[x])
                        print(" ")
                else:
                    print("No discovery engines defined for discovery server " + serverName) 
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        
        
def runDiscoveryService(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, discoveryRequestType, assetGUID):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/discovery-server/users/' + userId
        requestDiscoveryURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-request-types/' + discoveryRequestType + '/assets/' + assetGUID
        jsonHeader = {'content-type':'application/json'}
        body = {
            "class" : "DiscoveryRequestRequestBody",
        }
        response=requests.post(requestDiscoveryURL, json=body, headers=jsonHeader)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                reportGUID = response.json().get('guid')
                if reportGUID:
                    return reportGUID
                else:
                    print("No discovery returned from discovery server " + serverName) 
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def getDiscoveryReport(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, reportGUID):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/discovery-server/users/' + userId
        requestReportURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-analysis-reports/' + reportGUID
        response=requests.get(requestReportURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                printResponse(response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        
def getDiscoveryReportAnnotations(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, reportGUID):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/discovery-server/users/' + userId
        requestReportURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-analysis-reports/' + reportGUID + '/annotations?startingFrom=0&maximumResults=100'
        response=requests.get(requestReportURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                printResponse(response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


In [ ]:

#
# Perform basic checks to ensure the calling notebook has a good environment to work against.
# These are the only calls to run functions defined above.
#

print("\nChecking OMAG Server Platform availability...")

activatePlatform(corePlatformName, corePlatformURL, [cocoMDS2Name, cocoMDS3Name, cocoMDS5Name, cocoMDS6Name])
activatePlatform(dataLakePlatformName, dataLakePlatformURL, [cocoMDS1Name, cocoMDS4Name])
activatePlatform(devPlatformName, devPlatformURL, [cocoMDSxName])

print ("Done.")
print (" ")
